<a href="https://colab.research.google.com/github/krispy11tw/250416/blob/main/250430.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import urllib.request as req
import bs4 as bs

def save_post_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())
    # 作者/看版/標題/日期
    metas = html.find_all("span", {"class":"article-meta-value"})
    uid_text = metas[0].text
    board_text = metas[1].text
    title_text = metas[2].text
    time_text = metas[3].text
    # print(uid_text, board_text, title_text, time_text, sep="\n")
    # 每個推虛文 [{"type":"", "id":"", "text":"", "date_ip":""}]
    # JSON格式: {"左者": "看板": "表提": "推虛文":[]}
    push_list = []
    pushes = html.find_all("div", {"class":"push"})
    for push in pushes:
        push_meta = push.find_all("span")
        # 推噓處理
        push_type_text = push_meta[0].text
        if "推" in push_type_text:
            push_type_text = 1
        elif "噓" in push_type_text:
            push_type_text = -1
        else:
            push_type_text = 0
        # 推噓id
        push_id_text = push_meta[1].text
        # 推噓內容
        push_content_text = push_meta[2].text.replace(": ", "")
        # 推噓ipdate
        push_ipdate_text = push_meta[3].text.strip()
        # print(push_type_text, push_id_text, push_content_text, push_ipdate_text, sep="\t")
        push_data = {
            "type":push_type_text,
            "id":push_id_text,
            "text":push_content_text,
            "date_ip":push_ipdate_text
        }
        push_list.append(push_data)
    # print(push_list)
    post = {
        "作者":uid_text,
        "看板":board_text,
        "標題":title_text,
        "日期":time_text,
        "推噓文":push_list,
    }
    # print(post)
    fn = url.split("/")[-1] + ".json"
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(post, f, ensure_ascii=False, indent=4)

`````
當你發現你沒有class/id做篩選的時候
1. 找他老爸
2. 自己想辦法篩選
`````

In [ ]:
import json
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

allow_sub = ["jpg", "jpeg", "png", "gif"]
link_list = html.find_all("a")
for link in link_list:
    link_href = link["href"]
    sub = link_href.split(".")[-1]
    # lower是字串的專屬功能
    if sub.lower() in allow_sub:
        print(link_href)